In [ ]:
# 2등 코드 리뷰
# sampler와 ensemble을 이용.
# 저번에 잘못 말했는데 810000/2048은 나머지가 있다. coutinue_batch_sampler는 남기는 것 없이
# 하기위해서 drop_last를 False로 두고, 계속 이어서 하는것이다. 
# 2048/2048/.../46 이렇게 남더라. 그러면 46+2002/2048/2048/.../1072 이렇게 이어진다.

In [ ]:
# https://dacon.io/competitions/official/235554/codeshare/634?page=1&dtype=recent&ptype=pub

In [ ]:
!cat /proc/cpuinfo

In [ ]:
import time
import random
import numpy as np  # 1.18.1
from numpy.random import shuffle
import pandas as pd  # 0.25.3
import torch  # 1.4.0
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import TensorDataset
from torch.utils.data.sampler import Sampler, SequentialSampler

In [ ]:
# https://pytorch.org/docs/stable/notes/randomness.html
# deterministic 결정론적 알고리즘 - 특정 입력이 들어오면 학습했던 그대로.
# benchmark - cudnn이 결정론적 알고리즘을 선택하게 만들고. 성능을 유지시키는(REPRODUCIBILITY) 경향.
# 아래 코드는 seed 부터 REPRODUCIBILITY 까지. 가능하게 하기 위한 코드.

# https://hoya012.github.io/blog/reproducible_pytorch/

def set_seed(seed,mode=None):
    torch.manual_seed(seed)
    torch.backends.cudnn.deterministic=True
    torch.backends.cudnn.benchmark=False
    np.random.seed(seed)
    random.seed(seed)
    if mode=='reproductibility':
        torch.cuda.manual_seed(seed)
        torch.cuda.manual_seed_all(seed)
        
set_seed(71,mode='reproductibility')

In [ ]:
# custom_batch_sampler
# 1. sampler,batch_size,drop_last 가 필요하다. 
# 2. __init__,__iter__,__len__ 

epoch : 1000
batch_size : 2048
datasize : 810000

datasize / batch_size = 395.5078125 항상 남는게 있다.  

남는것을 batch로 해서 

In [ ]:
# https://pytorch.org/docs/stable/data.html
# __len__과 __iter__를 선언해야한다. class에선 __init__ 은 당연.

# https://subinium.github.io/pytorch-dataloader/ 
class ContinuousBatchSampler(Sampler):
    # https://hulk89.github.io/pytorch/2019/09/30/pytorch_dataset/
    # sampler를 상속받은 새로운 Batchsampler를 만든다. index_list를 반환한다.
    def __init__(self, sampler, batch_size, drop_last):
        # sequentialsampler 사용한다.0,1,2,3..810000 순서로 나온다.
        self.sampler = sampler
        self.batch_size = batch_size
        self.drop_last = drop_last # false 
        self.from_last_epoch = []

    def __iter__(self):
        # 같은 행이 걸리면 제외하고 drop_last가 false라 버리지 않고 뒤에 합쳐진다.
        idx_from_sampler = set(self.sampler)  # 0,1,2,3,4 810000... 에서
#         print(f"len(self.sampler) : {len(self.sampler)}")
#         print(f"len(self.from_last_epoch) : {len(self.from_last_epoch)}")
#         print(f"idx_from_sampler : {idx_from_sampler}")
        idx_to_exclude = set(self.from_last_epoch) # 배제할 idx
#         print(f"idx_to_exclude : {idx_to_exclude}")
        idx_after_exclusion = list(idx_from_sampler - idx_to_exclude)
        print(idx_after_exclusion[-5:])
#         print(f"idx_after_exclusion : {idx_after_exclusion}")
        shuffle(idx_after_exclusion) 
        first_batch = self.from_last_epoch + idx_after_exclusion[:self.batch_size - len(self.from_last_epoch)]
#         print(f"first_batch : {first_batch}")
        yield first_batch
    
        # 이부분은 이해가 안되는게 다시 붙이고 shuffle한다. 그러면 했던게 또 걸릴텐데.
        idx_of_left = sorted(idx_after_exclusion[self.batch_size - len(self.from_last_epoch):] + list(idx_to_exclude))
#         print(f"idx_of_left : {idx_of_left}")
        shuffle(idx_of_left)
        batch = []
        for idx in idx_of_left:
            batch.append(idx)
            if len(batch) == self.batch_size:
#                 print(f"batch : {batch}")
                yield batch
                batch = []
        # drop_last false로 뒀기 때문에 from_last_epoch에 이전 batch 를 계속 저장.
        if not self.drop_last:
            self.from_last_epoch = batch.copy()
#             print(f"from_last_epoch : {from_last_epoch}")
#         print(f"return : {(len(self.sampler) + len(self.from_last_epoch)) // self.batch_size}")
#         print(f"sampler lenght : {len(self.sampler)}")
#         print(f"from_last_epoch length : {len(self.from_last_epoch)}")

    def __len__(self):
        if self.drop_last:
            return len(self.sampler) // self.batch_size
        else:
#             print(f"return : {(len(self.sampler) + len(self.from_last_epoch)) // self.batch_size}")
#             print(f"sampler lenght : {len(self.sampler)}")
#             print(f"from_last_epoch length : {len(self.from_last_epoch)}")
            return (len(self.sampler) + len(self.from_last_epoch)) // self.batch_size


### 결론

# 1. 처음 한것은 batch가 끝나고 뒤에 붙인다 -> drop_last=False 이므로
# 2. yield 때문인지는 몰라도 first_batch 이후 batch가 나오게 되면서, 처음 len(self.sampler) // self.batch_size 이후,
#  (len(self.sampler) + len(self.from_last_epoch)) // self.batch_size 게 된다. 
# -> 이부분은 설명을 잘 못하겠습니다. 느낌입니다. print 끼고 하려니까 안나와서요. 훈련과정이 눈에 안보입니다.
# 결국, dynamic batch 느낌입니다. 보통 batch는 2048 2048 순서적이라면
# 이거 읽어보려구요. https://discuss.pytorch.org/t/dataloader-for-variable-batch-size/13840

In [ ]:
device = torch.device('cuda:0') # 혹시 이 뒤에 숫자가 의미하는게 무엇인지 알고 있는분 계신가요?
num_epochs = 1000
batch_size = 2048
initial_learning_rate = 0.2
# https://pytorch.org/docs/stable/data.html
# which enables fast data transfer to CUDA-enabled GPUs.
loader_params = {'num_workers': 0, 'pin_memory': True}

train_data = np.array(pd.read_csv('../input/month1/data_mdc01/train.csv'), dtype=np.float32)
X_train = torch.tensor(train_data[:, 4:], dtype=torch.float32)
y_train = torch.tensor(train_data[:, :4], dtype=torch.float32)

# dataset은 그냥 tensor 형으로 해주는것 같고.
dataset = TensorDataset(X_train, y_train)
# dataloaders를 정의해야한다. dataset, batch_sampler,loader_params
train_loader = torch.utils.data.DataLoader(dataset, batch_sampler=ContinuousBatchSampler(
    sampler=SequentialSampler(range(len(dataset))), batch_size=batch_size, drop_last=False), **loader_params)

prediction = np.zeros((10000, 4), dtype=np.float32)
logging_term = 1000
logging_total = int(810000 * num_epochs / batch_size)

In [ ]:
net = nn.Sequential(
        nn.BatchNorm1d(226),
        nn.ReLU(),
        nn.Linear(226, 768),
        nn.BatchNorm1d(768),
        nn.ReLU(),
        nn.Linear(768, 768),
        nn.BatchNorm1d(768),
        nn.ReLU(),
        nn.Linear(768, 768),
        nn.BatchNorm1d(768),
        nn.ReLU(),
        nn.Linear(768, 768),
        nn.BatchNorm1d(768),
        nn.ReLU(),
        nn.Linear(768, 768),
        nn.BatchNorm1d(768),
        nn.ReLU(),
        nn.Linear(768, 768),
        nn.BatchNorm1d(768),
        nn.ReLU(),
        nn.Linear(768, 4)
    )

model = net.to(device)
running_loss = 0.
running_counter = 0
criterion = torch.nn.L1Loss()
# momentum 은 운동 관성. 관성을 넣어 학습속도를 제어한다고 한다.
# 학습 2에서는 weight_decay가 있는데 l2 loss라고 한다.
# https://deepapple.tistory.com/6 - loss를 더해서 모델의 복잡도를 낮춘다.
optimizer = optim.SGD(model.parameters(), lr=initial_learning_rate, momentum=0.9)

# https://sanghyu.tistory.com/113
# parameter에서 cosine annealing 함수를 따르고, restart는 250개의 iter가끝나고 상황보고
# T_mult restart 후에 T_i를 증가시키는 factor 라는데 뜻은 모르겠다.
# eta_min은 최소 lr ,last epoch 는 -1이 default다. 
scheduler = optim.lr_scheduler.CosineAnnealingWarmRestarts(optimizer, T_0=250, T_mult=1, eta_min=0.005,
                                                           last_epoch=-1)
# 1 epoch 당 395번 돌리고, 
for epoch in range(num_epochs):
    # set mode 
    model.train()
    for xx, yy in train_loader:
        
        # gpu 계산으로.
        xx, yy = xx.to(device), yy.to(device)
        # gradient 계산을 불가능하게 하는것.
        with torch.no_grad():
            # noise를 더한다.
            xx += torch.randn((xx.shape[0], 226), device='cuda:0') * 0.003

        out = model(xx)
        loss = criterion(out, yy)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        running_counter += 1
        running_loss += loss.item()

        if running_counter % logging_term == 0:
            print(f'epoch {epoch} (iter {running_counter:6d}/{logging_total:6d})')
            running_loss = 0.
    # epoch 당 한번 scheduler로 learning_rate 조절.
    # 원래 epoch 끝나고 scheduler로 learning_rate 조절.
    scheduler.step()
    # 이렇게 한 이유는 epoch에 따라 어떻게 다른지 확인. 위에서 continuesampler 만들때
    # yield가 두번 들어가는데 yield는 몇번 들어가도 상관없고, first batch 보낸 이후에 idx_left_batch 계속 보낸다.
model.eval()
output = model(
    torch.tensor(np.array(pd.read_csv('../input/month1/data_mdc01/test.csv'), dtype=np.float32))[:, 1:].to(device))
output = np.array(output.detach().to('cpu'), dtype=np.float32)
# 학습2,3,4에서는 1/7 모델을 20개 해서 0.05, 7개중의 1개라 1/7 앙상블을 뜻한다.
# prediction += output 

In [ ]:
class net(nn.Module):
    def __init__(self):
        # class 변수가져오기
        super.__init__()
#         https://pytorch.org/docs/stable/generated/torch.nn.AvgPool1d.html
        self.avg=nn.AvgPool1d(kernel_size=3,stride=3,padding=1)
        self.linear=nn.Sequential(
            nn.Linear(76, 768),
            nn.BatchNorm1d(768),
            nn.ReLU(),
            nn.Linear(768, 768),
            nn.BatchNorm1d(768),
            nn.ReLU(),
            nn.Linear(768, 768),
            nn.BatchNorm1d(768),
            nn.ReLU(),
            nn.Linear(768, 768),
            nn.BatchNorm1d(768),
            nn.ReLU(),
            nn.Linear(768, 768),
            nn.BatchNorm1d(768),
            nn.ReLU(),
            nn.Linear(768, 768),
            nn.BatchNorm1d(768),
            nn.ReLU(),
            nn.Linear(768, 768),
            nn.BatchNorm1d(768),
            nn.ReLU(),
            nn.Linear(768, 768),
            nn.BatchNorm1d(768),
            nn.ReLU(),
            nn.Linear(768, 4)
        )
        def forward(self,x):
            # N? channel input
            # 1,1,226 -> 1,1,76 -> 1,76 -> 1,768 ... -> 1,4
            x=x.view(x.shape[0],1,-1)
            x=self.avg(x)
            x=x.view(x.shape[0],-1)
            x=self.linear(x)
            return x

In [ ]:
# https://wegonnamakeit.tistory.com/47
# batch에 있는 데이터가 변화하더라도 normalize하는 mean과 varianece 값이 바뀌지 않게 된다.

# 이게 중요한것 같다. 데이터가 변해도 mean과 variance 가 그대로면 정해진 범위안에서 노는것이다. 

for model_no in ['Model_01', 'Model_02', 'Model_03', 'Model_04', 'Model_05',
                 'Model_06', 'Model_07', 'Model_08', 'Model_09', 'Model_10',
                 'Model_11', 'Model_12', 'Model_13', 'Model_14', 'Model_15',
                 'Model_16', 'Model_17', 'Model_18', 'Model_19', 'Model_20']:

    net = nn.Sequential(
        nn.BatchNorm1d(226),
        nn.ReLU(),
        nn.Linear(226, 768),
        nn.BatchNorm1d(768),
        nn.ReLU(),
        nn.Linear(768, 768),
        nn.BatchNorm1d(768),
        nn.ReLU(),
        nn.Linear(768, 768),
        nn.BatchNorm1d(768),
        nn.ReLU(),
        nn.Linear(768, 768),
        nn.BatchNorm1d(768),
        nn.ReLU(),
        nn.Linear(768, 768),
        nn.BatchNorm1d(768),
        nn.ReLU(),
        nn.Linear(768, 768),
        nn.BatchNorm1d(768),
        nn.ReLU(),
        nn.Linear(768, 4)
    )

    model = net.to(device)
    running_loss = 0.
    running_counter = 0
    criterion = torch.nn.L1Loss()
    # momentum 은 운동 관성. 관성을 넣어 학습속도를 제어한다고 한다.
    # 학습 2에서는 weight_decay가 있는데 l2 loss라고 한다.
    # https://deepapple.tistory.com/6 - loss를 더해서 모델의 복잡도를 낮춘다.
    optimizer = optim.SGD(model.parameters(), lr=initial_learning_rate, momentum=0.9)
    
    # https://sanghyu.tistory.com/113
    # parameter에서 cosine annealing 함수를 따르고, restart는 250개의 iter가끝나고 상황보고
    # T_mult restart 후에 T_i를 증가시키는 factor 라는데 뜻은 모르겠다.
    # eta_min은 최소 lr ,last epoch 는 -1이 default다. 
    scheduler = optim.lr_scheduler.CosineAnnealingWarmRestarts(optimizer, T_0=250, T_mult=1, eta_min=0.005,
                                                               last_epoch=-1)
    count_a=0
    for epoch in range(num_epochs):

        model.train()
        count_b=0
        for xx, yy in train_loader:
            count_b+=1
            # gpu 계산으로.
            xx, yy = xx.to(device), yy.to(device)
            # gradient 계산을 불가능하게 하는것.
            with torch.no_grad():
                # noise를 더한다.
                xx += torch.randn((xx.shape[0], 226), device='cuda:0') * 0.003

            out = model(xx)
            loss = criterion(out, yy)

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            running_counter += 1
            running_loss += loss.item()
            
            # 한 epoch 당 1000번 학습
            if running_counter % logging_term == 0:
                print(model_no + ' (iter {:6d}/{:6d}) {:.4f}'.format(running_counter, logging_total,
                                                                     running_loss / logging_term))
                running_loss = 0.
            if count_b==2:
                break
        # epoch 당 한번 scheduler로 learning_rate 조절.
        # 원래 epoch 끝나고 scheduler로 learning_rate 조절.
        scheduler.step()
        # 이렇게 한 이유는 epoch에 따라 어떻게 다른지 확인. 위에서 continuesampler 만들때
        # yield가 두번 들어가는데 yield는 몇번 들어가도 상관없고, first batch 보낸 이후에 idx_left_batch 계속 보낸다.
        if count_a==2:
            break
    break
    model.eval()
    output = model(
        torch.tensor(np.array(pd.read_csv('../input/month1/data_mdc01/test.csv'), dtype=np.float32))[:, 1:].to(device))
    output = np.array(output.detach().to('cpu'), dtype=np.float32)
    # 학습2,3,4에서는 1/7 모델을 20개 해서 0.05, 7개중의 1개라 1/7 앙상블을 뜻한다.
    prediction += output * 0.05
    
    # 모델 하나당 시간이 엄청 걸린다. 

In [ ]:
# 1등 코드리뷰
# scheduler 로 learning_rate에서 loss가 줄면 조금씩 변형하는 scheduler 사용.
# with torch.no_grad(): 로 validation때 gradient 계산을 안하는것으로. 이렇게 해야하는것같다.
# <> 형태의 복잡한 mlp 형태. skip-connection이 뭐 안하고 뛰어넘는다가 아니라, 뛰어넘어서 예전거도 기억하게 하는것같다.

In [ ]:
import math
import time
from itertools import chain
import argparse
import numpy as np
import pandas as pd
import torch 
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from tqdm.auto import tqdm
from torch.optim import Optimizer
from torch.optim.lr_scheduler import LambdaLR
# 모델 학습을 위해 CUDA 환경 설정. : 지피유 설정
device = torch.device('cuda:0' if torch.cuda.is_available else 'cpu')

In [ ]:
device = torch.device('cuda:0' if torch.cuda.is_available else 'cpu')

In [ ]:
# 별도의 데이터 Pre-Processing 과정은 없고 모델 훈련시 검증을 위해 train 중 10000개를 validation 용으로 분리.
# 새로 만든 train.csv는 train1.csv, validation은 val.csv로 저장.
# dataframe.sample(frac=1) 을 통해 셔플.
from IPython.display import display

path_train = '../input/month1/data_mdc01/train.csv'
path_test = '../input/month1/data_mdc01/test.csv'
layers = [['layer_1','layer_2','layer_3','layer_4'], [str(i) for i in np.arange(0,226).tolist()]]
layers = list(chain(*layers))
# display(len(layers))

train = pd.read_csv(path_train)
# print(train.shape)
train = train.sample(frac=1)
rows, cols = train.shape

train1 = train.iloc[:rows - 10000,:]
train1 = train1.values
train1 = pd.DataFrame(data=train1,columns=layers)
# display(train1)

train1.to_csv('train1.csv', index_label='id')

print("train file saved....")
val = train.iloc[rows - 10000:,:]
val = val.values
val = pd.DataFrame(data=val,columns=layers)
val.to_csv('val.csv', index_label='id')
# display(val)

In [ ]:
# 새로 만든 train/ val 모델 학습 데이터 경로를 설정.
train_path = './train1.csv'
val_path = './val.csv'

lr = 1e-03
adam_epsilon = 1e-06
epochs = 100
batch_size = 2048
warmup_step = 2000
loss_fn = nn.L1Loss()

class PandasDataset(Dataset):
    def __init__(self, path):
        super(PandasDataset, self).__init__()
        train = pd.read_csv(path).iloc[:,1:]
        self.train_X, self.train_Y = train.iloc[:,4:], train.iloc[:,0:4]
        self.tmp_x , self.tmp_y = self.train_X.values, self.train_Y.values
    
    def __len__(self):
        return len(self.train_X)

    def __getitem__(self, idx):
        return {
            'X':torch.from_numpy(self.tmp_x)[idx],
            'Y':torch.from_numpy(self.tmp_y)[idx]
        }
            
train_dataset = PandasDataset(train_path)
train_loader = DataLoader(train_dataset, batch_size=batch_size,  num_workers=4)

val_dataset = PandasDataset(val_path)
val_loader = DataLoader(val_dataset, batch_size=batch_size,  num_workers=4) 

sigmoid = $1\over{1+e^{-z}}$   
tanh = $2 sigmoid(z) -1$  
gelu = $1\over2$ $x$ ($\sqrt{1+{2\over\pi}}$) ($ x + 0.044715 * x^3 $ )  


https://arxiv.org/abs/1606.08415   
https://medium.com/@shoray.goel/gelu-gaussian-error-linear-unit-4ec59fb2e47c  

Gaussian Error Linear Unit (GELUs) 라 불리고, relu나 elu에 비해 잘 작동한다고 나와있다.  
보통 computer vision 에 많이 쓰이는것같다. 

In [ ]:
# activation
class GELU(nn.Module):
    def forward(self, x):
        return 0.5 * x * (1 + torch.tanh(math.sqrt(2 / math.pi) * (x + 0.044715 * torch.pow(x, 3))))
    
# layer 정규화
class LayerNorm(nn.Module):
    def __init__(self, hidden_size, eps=1e-5):
        """
        Construct a layernorm module in the TF style (epsilon inside the square root).
        """
        super(LayerNorm, self).__init__()
        self.weight = nn.Parameter(torch.ones(hidden_size))
        self.bias = nn.Parameter(torch.zeros(hidden_size))
        self.variance_epsilon = eps

        self.init_weights()

    def init_weights(self):
        self.weight.data.fill_(1.0)
        self.bias.data.zero_()

    def forward(self, x):
        u = x.mean(-1, keepdim=True)
        s = (x - u).pow(2).mean(-1, keepdim=True)
        x = (x - u) / torch.sqrt(s + self.variance_epsilon)
        return self.weight * x + self.bias

In [ ]:
# model
class skipConnectionModel(nn.Module):
    def __init__(self):
        super(skipConnectionModel, self).__init__()
        
        self.ln = LayerNorm(10000)
        self.ln1 = LayerNorm(7000)
        self.ln2 = LayerNorm(4000)
        self.ln3 = LayerNorm(2000)
        
        self.upblock1 = nn.Sequential(nn.Linear(226, 2000),GELU(),nn.BatchNorm1d(2000))
        self.upblock2 = nn.Sequential(nn.Linear(2000,4000),GELU(),nn.BatchNorm1d(4000))
        self.upblock3 = nn.Sequential(nn.Linear(4000,7000), GELU(),nn.BatchNorm1d(7000))
        self.upblock4 = nn.Sequential(nn.Linear(7000,10000),GELU(),nn.BatchNorm1d(10000))

        self.downblock1 = nn.Sequential(nn.Linear(10000, 7000),GELU(),nn.BatchNorm1d(7000))
        self.downblock2 = nn.Sequential(nn.Linear(7000, 4000),GELU(),nn.BatchNorm1d(4000))
        self.downblock3 = nn.Sequential(nn.Linear(4000, 2000),GELU(),nn.BatchNorm1d(2000))
        self.downblock4 = nn.Sequential(nn.Linear(2000, 300),GELU(),nn.BatchNorm1d(300))
        
        self.fclayer = nn.Sequential(nn.Linear(300,4))
        self.dropout = nn.Dropout(0.1)
        
    def forward(self, x):
        upblock1_out = self.upblock1(x)
        upblock2_out = self.upblock2(upblock1_out)
        upblock3_out = self.upblock3(upblock2_out)
        upblock4_out = self.upblock4(upblock3_out)
        
        downblock1_out = self.downblock1(self.ln(upblock4_out))
        skipblock1 = downblock1_out + upblock3_out
        downblock2_out = self.downblock2(self.ln1(skipblock1))
        skipblock2 = downblock2_out + upblock2_out
        downblock3_out = self.downblock3(self.ln2(skipblock2))
        skipblock3 = downblock3_out + upblock1_out
        downblock4_out = self.downblock4(self.ln3(skipblock3))
        
        output = self.fclayer(downblock4_out)
        
        return output

In [ ]:
def get_constant_schedule(optimizer, last_epoch=-1):
    """ Create a schedule with a constant learning rate.
    """
    return LambdaLR(optimizer, lambda _: 1, last_epoch=last_epoch)

def get_constant_schedule_with_warmup(optimizer, num_warmup_steps, last_epoch=-1):
    """ Create a schedule with a constant learning rate preceded by a warmup
    period during which the learning rate increases linearly between 0 and 1.
    """

    def lr_lambda(current_step):
        if current_step < num_warmup_steps:
            return float(current_step) / float(max(1.0, num_warmup_steps))
        return 1.0

    return LambdaLR(optimizer, lr_lambda, last_epoch=last_epoch)

def get_linear_schedule_with_warmup(optimizer, num_warmup_steps, num_training_steps, last_epoch=-1):
    """ Create a schedule with a learning rate that decreases linearly after
    linearly increasing during a warmup period.
    """

    def lr_lambda(current_step):
        if current_step < num_warmup_steps:
            return float(current_step) / float(max(1, num_warmup_steps))
        return max(
            0.0, float(num_training_steps - current_step) / float(max(1, num_training_steps - num_warmup_steps))
        )

    return LambdaLR(optimizer, lr_lambda, last_epoch)

def get_cosine_schedule_with_warmup(optimizer, num_warmup_steps, num_training_steps, num_cycles=0.5, last_epoch=-1):
    """ Create a schedule with a learning rate that decreases following the
    values of the cosine function between 0 and `pi * cycles` after a warmup
    period during which it increases linearly between 0 and 1.
    """

    def lr_lambda(current_step):
        if current_step < num_warmup_steps:
            return float(current_step) / float(max(1, num_warmup_steps))
        progress = float(current_step - num_warmup_steps) / float(max(1, num_training_steps - num_warmup_steps))
        return max(0.0, 0.5 * (1.0 + math.cos(math.pi * float(num_cycles) * 2.0 * progress)))

    return LambdaLR(optimizer, lr_lambda, last_epoch)

def get_cosine_with_hard_restarts_schedule_with_warmup(
    optimizer, num_warmup_steps, num_training_steps, num_cycles=1.0, last_epoch=-1
):
    """ Create a schedule with a learning rate that decreases following the
    values of the cosine function with several hard restarts, after a warmup
    period during which it increases linearly between 0 and 1.
    """

    def lr_lambda(current_step):
        # 2000으로 시작해서 current_step이 2000을 넘으면
        # 810000//2048=395
        # num_training_step = len(train_loader) * epochs = 보통 388 * 30 > 10000
        if current_step < num_warmup_steps:
            return float(current_step) / float(max(1, num_warmup_steps))
        # progress= 20xx-2048/10000 ? 
        progress = float(current_step - num_warmup_steps) / float(max(1, num_training_steps - num_warmup_steps))
        if progress >= 1.0:
            return 0.0
        # 
        return max(0.0, 0.5 * (1.0 + math.cos(math.pi * ((float(num_cycles) * progress) % 1.0))))

    return LambdaLR(optimizer, lr_lambda, last_epoch)

max (0, $1 \over 2$ ($1+ \cos (\pi * {progress})$))

In [ ]:
class AdamW(Optimizer):
    """ Implements Adam algorithm with weight decay fix.
    Parameters:
        lr (float): learning rate. Default 1e-3.
        betas (tuple of 2 floats): Adams beta parameters (b1, b2). Default: (0.9, 0.999)
        eps (float): Adams epsilon. Default: 1e-6
        weight_decay (float): Weight decay. Default: 0.0
        correct_bias (bool): can be set to False to avoid correcting bias in Adam (e.g. like in Bert TF repository). Default True.
    """

    def __init__(self, params, lr=1e-3, betas=(0.9, 0.999), eps=1e-6, weight_decay=0.0, correct_bias=True):
        if lr < 0.0:
            raise ValueError("Invalid learning rate: {} - should be &gt;= 0.0".format(lr))
        if not 0.0 <= betas[0] < 1.0:
            raise ValueError("Invalid beta parameter: {} - should be in [0.0, 1.0[".format(betas[0]))
        if not 0.0 <= betas[1] < 1.0:
            raise ValueError("Invalid beta parameter: {} - should be in [0.0, 1.0[".format(betas[1]))
        if not 0.0 <= eps:
            raise ValueError("Invalid epsilon value: {} - should be &gt;= 0.0".format(eps))
        defaults = dict(lr=lr, betas=betas, eps=eps, weight_decay=weight_decay, correct_bias=correct_bias)
        super().__init__(params, defaults)

    def step(self, closure=None):
        """Performs a single optimization step.
        Arguments:
            closure (callable, optional): A closure that reevaluates the model
                and returns the loss.
        """
        loss = None
        if closure is not None:
            loss = closure()

        for group in self.param_groups:
            for p in group["params"]:
                if p.grad is None:
                    continue
                grad = p.grad.data
                if grad.is_sparse:
                    raise RuntimeError("Adam does not support sparse gradients, please consider SparseAdam instead")

                state = self.state[p]

                # State initialization
                if len(state) == 0:
                    state["step"] = 0
                    # Exponential moving average of gradient values
                    state["exp_avg"] = torch.zeros_like(p.data)
                    # Exponential moving average of squared gradient values
                    state["exp_avg_sq"] = torch.zeros_like(p.data)

                exp_avg, exp_avg_sq = state["exp_avg"], state["exp_avg_sq"]
                beta1, beta2 = group["betas"]

                state["step"] += 1

                # Decay the first and second moment running average coefficient
                # In-place operations to update the averages at the same time
                exp_avg.mul_(beta1).add_(1.0 - beta1, grad)
                exp_avg_sq.mul_(beta2).addcmul_(1.0 - beta2, grad, grad)
                denom = exp_avg_sq.sqrt().add_(group["eps"])

                step_size = group["lr"]
                if group["correct_bias"]:  # No bias correction for Bert
                    bias_correction1 = 1.0 - beta1 ** state["step"]
                    bias_correction2 = 1.0 - beta2 ** state["step"]
                    step_size = step_size * math.sqrt(bias_correction2) / bias_correction1

                p.data.addcdiv_(-step_size, exp_avg, denom)

                # Just adding the square of the weights to the loss function is *not*
                # the correct way of using L2 regularization/weight decay with Adam,
                # since that will interact with the m and v parameters in strange ways.
                #
                # Instead we want to decay the weights in a manner that doesn't interact
                # with the m/v parameters. This is equivalent to adding the square
                # of the weights to the loss with plain (non-momentum) SGD.
                # Add weight decay at the end (fixed version)
                if group["weight_decay"] > 0.0:
                    p.data.add_(-group["lr"] * group["weight_decay"], p.data)

        return loss   

In [ ]:
# model = skipConnectionModel()
model = model.to(device) # 모델을 GPU 메모리에 올림.

In [ ]:
"""
모델 학습
"""

total_step = len(train_loader) * epochs
print(f"Total step is....{total_step}") # 모델이 학습하는 전체 step 계산.

# 옵티마이저와 스케줄러의 파라미터들을 정의.

no_decay = ["bias", "LayerNorm.weight"] # decay하지 않을 영역 지정.
optimizer_grouped_parameters = [
    {
        "params": [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)],
        "weight_decay": 0.0,
    },
    {"params": [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], "weight_decay": 0.0},
]
optimizer = AdamW(optimizer_grouped_parameters, lr=lr, eps=adam_epsilon)
scheduler = get_cosine_with_hard_restarts_schedule_with_warmup(
    optimizer, num_warmup_steps=warmup_step, num_training_steps=total_step
)

# train loss와 val loss 지정.
total_loss = 0.0
total_val_loss = 0.0

# 모델 이름을 위해서 변수 만듦.
version = time.localtime()[3:5]
curr_lr = lr

n_val_loss = 10000000. # 가장 낮은 validation loss를 저장하기 위해서 변수 설정.

for epoch in range(epochs):
    total_loss = 0 
    total_val_loss = 0
    for i, data in enumerate(tqdm(train_loader, desc='*********Train mode*******')):  # train 데이터를 부르고 학습.
        # forward pass
        pred = model(data['X'].float().to(device))
        loss = loss_fn(pred, data['Y'].float().to(device))
        
        # backward pass
        optimizer.zero_grad() # optimizer 객체 사용해서 학습 가능한 가중치 변수에 대한 모든 변화도를 0으로 만듦
        loss.backward() 
        optimizer.step() # update optimizer params 
        # 순서 중요.
        scheduler.step() # update scheduler params
        
        total_loss += loss.item()
        
    train_loss = total_loss / len(train_loader)
    print ("Epoch [{}/{}], Train Loss: {:.4f}".format(epoch+1, epochs, train_loss))

    # evaluation
    # validation 데이터를 부르고 epoch 마다 학습된 모델을 부르고 평가.
    model.eval()
    with torch.no_grad():
        for i, data in enumerate(tqdm(val_loader, desc='*********Evaluation mode*******')):
            pred = model(data['X'].float().to(device))
            loss_val = loss_fn(pred, data['Y'].float().to(device))
            
            total_val_loss += loss_val.item()
    val_loss = total_val_loss / len(val_loader)
    print ("Epoch [{}/{}], Eval Loss: {:.4f}".format(epoch+1, epochs, val_loss))
    
    # best model을 저장.
    if val_loss < n_val_loss:
        n_val_loss = val_loss
        torch.save(model.state_dict(), f'test_{version}_{lr}_{epochs}.pth')
        print("Best Model saved......")

In [ ]:
"""
모델 테스트
"""

test_model = skipConnectionModel()

# test 파일 경로 및 test 데이터 로드
path_test = 'test.csv'
class TestDataset(Dataset):
    def __init__(self, path_test):
        super(TestDataset, self).__init__()
        test = pd.read_csv(path_test)
        self.test_X = test.iloc[:,1:]
        self.tmp_x = self.test_X.values
    
    def __len__(self):
        return len(self.test_X)

    def __getitem__(self, idx):
        return torch.from_numpy(self.tmp_x)[idx]
    
test_data = TestDataset(path_test)
test_loader = DataLoader(test_data, batch_size=10000,  num_workers=4)

# 모델에 학습된 가중치를 업로드.
weights = torch.load(f'test_{version}_{lr}_{epochs}.pth', map_location='cuda:0')
test_model.load_state_dict(weights)
test_model = test_model.to(device)
test_model.eval()

with torch.no_grad():
    for data in test_loader:
        data = data.to(device)
        outputs = test_model(data.float())
pred_test = outputs

sample_sub = pd.read_csv('sample_submission.csv', index_col=0)
layers = ['layer_1','layer_2','layer_3','layer_4']
submission = sample_sub.values + pred_test.cpu().numpy()

submission = pd.DataFrame(data=submission,columns=layers)
submission.to_csv(f'test_{version}_{lr}_{epochs}.csv', index_label='id')

In [ ]:
# + Self evaluation and Ensemble
# Dacon의 제출하기를 통해 측정한 mae값 중 가장 낮은 mae csv 파일과 다른 파라미터 적용으로 훈련한 모델과 mae 비교를 통해
# 대략적인 test mae를 예상한 뒤 제출하기 하여 3번 제출할 수 있는 기회를 최대한 살림.
# 다양한 파라미터 적용을 통한 모델들을 아래의 en함수를 통해 합친 뒤 평균을 구하여 제출
# (추가하는 csv 파일의 수에 따라 en함수의 함수가 받는 csv 파일 개수 증가 및 코드 수정 필요.)
def mae(best_path, my_path):
    best = pd.read_csv(best_path)
    best_value = best.iloc[:,1:].values
    value = pd.read_csv(my_path)
    my_value = value.iloc[:,1:].values
    abs_value = abs(best_value - my_value)
    size = abs_value.shape
    return sum(sum(abs_value) / (size[0]*size[1]))

def en(best_path, my_path):
    best = pd.read_csv(best_path)
    best_value = best.iloc[:,1:].values
    value = pd.read_csv(my_path)
    my_value = value.iloc[:,1:].values
    return (my_value + best_value)/2

In [ ]:
import os
import gc
import numpy as np
import pandas as pd
import math
import random
from time import time
from tqdm import tqdm
from pathlib import Path

import torch
from torch import nn, cuda
import torchvision.models as models
from torch.utils.data import DataLoader, Dataset

from torch.optim import Adam, SGD, Optimizer
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

import warnings
warnings.filterwarnings('ignore')

In [ ]:
def set_seed(seed,mode=None):
    torch.manual_seed(seed)
    torch.backends.cudnn.deterministic=True
    torch.backends.cudnn.benchmark=False
    np.random.seed(seed)
    random.seed(seed)
    if mode=='reproductibility':
        torch.cuda.manual_seed(seed)
        torch.cuda.manual_seed_all(seed)
        
set_seed(71)

In [ ]:
class Semi_dataset(Dataset):
    # custom dataset은 __init__,__len__,__getitem__ 가 필요.
    # https://wikidocs.net/57165
    # tensor 형으로 바꿔야 한다.
    def __init__(self, X, Y):
        self.X = X
        self.Y = Y
        self.X_dataset = []
        self.Y_dataset = []
        for x in X:
            # tensor형으로 바꾸고 
            self.X_dataset.append(torch.FloatTensor(x))
        try:
            for y in Y.values:
                self.Y_dataset.append(torch.tensor(y))
        except:
            print("no label")
            
    def __len__(self):
        return len(self.X)

    def __getitem__(self, index):
        # tensor 형을 내보내고 index에 따라서
        data = self.X_dataset[index]
        try:
            target = self.Y_dataset[index]
            return data, target
        except:
            return data

In [ ]:
def build_dataloader(X, Y, batch_size, shuffle=False):
    # custom dataloader
    dataset = Semi_dataset(X, Y)
    dataloader = DataLoader(
                            # custom dataset
                            dataset,
                            batch_size=batch_size,
                            shuffle=shuffle,
                            # cpu 갯수 잘 확인할것. 이걸로 error 많이 남. 0으로 두는데 그럼 느리다.
                            num_workers=4
                            )
    return dataloader

# 왜 이렇게 돌아가는지 모르겠다.
# 평균절대오차를 구현했다.
from sklearn.metrics import mean_absolute_error as mae

def mean_absolute_error(y_true, y_pred,
                        sample_weight=None,
                        multioutput='uniform_average'):
    
    output_errors = np.average(np.abs(y_pred - y_true),
                               weights=sample_weight, axis=0)
    if isinstance(multioutput, str):
        if multioutput == 'raw_values':
            return output_errors
        elif multioutput == 'uniform_average':
            multioutput = None

    return np.average(output_errors, weights=multioutput)

In [ ]:
from sklearn.metrics import mean_absolute_error as mae
from IPython.display import display

a=np.array([1,2,3,4,5])
b=np.array([1.5,2.5,2.5,4.5,6.5])

display(mean_absolute_error(a,b))
display(mae(a,b))

In [ ]:
class AdamW(Optimizer):
    def __init__(self, params, lr=1e-3, betas=(0.9, 0.999), eps=1e-8,
                 weight_decay=0):
        defaults = dict(lr=lr, betas=betas, eps=eps,
                        weight_decay=weight_decay)
        super(AdamW, self).__init__(params, defaults)

    def step(self, closure=None):
        """Performs a single optimization step.

        Arguments:
            closure (callable, optional): A closure that reevaluates the model
                and returns the loss.
        """
        loss = None
        if closure is not None:
            loss = closure()

        for group in self.param_groups:
            for p in group['params']:
                if p.grad is None:
                    continue
                grad = p.grad.data
                if grad.is_sparse:
                    raise RuntimeError('AdamW does not support sparse gradients, please consider SparseAdam instead')

                state = self.state[p]

                # State initialization
                if len(state) == 0:
                    state['step'] = 0
                    # Exponential moving average of gradient values
                    state['exp_avg'] = torch.zeros_like(p.data)
                    # Exponential moving average of squared gradient values
                    state['exp_avg_sq'] = torch.zeros_like(p.data)

                exp_avg, exp_avg_sq = state['exp_avg'], state['exp_avg_sq']
                beta1, beta2 = group['betas']

                state['step'] += 1

                # Decay the first and second moment running average coefficient
                exp_avg.mul_(beta1).add_(1 - beta1, grad)
                exp_avg_sq.mul_(beta2).addcmul_(1 - beta2, grad, grad)

                denom = exp_avg_sq.sqrt().add_(group['eps'])

                bias_correction1 = 1 - beta1 ** state['step']
                bias_correction2 = 1 - beta2 ** state['step']
                step_size = group['lr'] * math.sqrt(bias_correction2) / bias_correction1

                p.data.addcdiv_(-step_size, exp_avg, denom)

                if group['weight_decay'] != 0:
                    p.data.add_(-group['weight_decay'], p.data)

        return loss

In [ ]:
class MLP_only_flatfeatures(nn.Module):
    def __init__(self, num_classes=1):
        super(MLP_only_flatfeatures, self).__init__()
        self.num_classes = num_classes         
        self.fc_layers = nn.Sequential(
            nn.Linear(226, 1000),
            nn.ReLU(),
            nn.BatchNorm1d(1000),
            nn.Linear(1000, 500),
            nn.ReLU(),
            nn.BatchNorm1d(500),
            nn.Dropout(0.1),
            nn.Linear(500, 128),
            nn.ReLU(),
            nn.BatchNorm1d(128),
            nn.Dropout(0.1),
            nn.Linear(128, self.num_classes)
            )             
        self._initialize_weights()

    def forward(self, x):
        out = self.fc_layers(x)
        return out

    def _initialize_weights(self):
        for m in self.modules():
            if isinstance(m, nn.Linear):
                nn.init.normal_(m.weight, 0, 0.01)
                nn.init.constant_(m.bias, 0)

In [ ]:
def build_model(device, model_name='mlp', weight_path=None):

    if model_name == 'mlp':
        model = MLP_only_flatfeatures(4)
    
    return model

In [ ]:
def validation(model, criterion, valid_loader, device):
    
    model.eval()
    valid_preds = np.zeros((len(valid_loader.dataset), 4))
    valid_targets = np.zeros((len(valid_loader.dataset), 4))
    val_loss = 0.
    
    with torch.no_grad():
        for i, (data, target) in enumerate(valid_loader):
            
            valid_targets[i * batch_size: (i+1) * batch_size] = target.float().numpy().copy()

            data = data.to(device)
            target = target.float().to(device)
                
            output = model(data)
            loss = criterion(output, target)
            
            valid_preds[i * batch_size: (i+1) * batch_size] = output.detach().cpu().numpy()
            
            val_loss += loss.item() / len(valid_loader)
    
    val_score = mean_absolute_error(valid_preds, valid_targets)

    return val_loss, val_score

In [ ]:
if cuda.is_available:
    device = torch.device("cuda:0")
else:
    device = torch.device("cpu")

In [ ]:
DATASET_PATH = '../input/month1/data_mdc01'
train_df = pd.read_csv(os.path.join(DATASET_PATH, 'train.csv'))
test_df = pd.read_csv(os.path.join(DATASET_PATH, 'test.csv'))
submission_df = pd.read_csv(os.path.join(DATASET_PATH, 'sample_submission.csv'))

In [ ]:
X_train, X_val, y_train, y_val = train_test_split(train_df.iloc[:, 4:], train_df.iloc[:, :4], test_size=0.02, random_state=71, shuffle=True)

In [ ]:
X_train = X_train.reset_index(drop=True)
X_val = X_val.reset_index(drop=True)
y_train = y_train.reset_index(drop=True)
y_val = y_val.reset_index(drop=True)

In [ ]:
scaler = StandardScaler()
scaler.fit(X_train)
X_train = scaler.transform(X_train)
X_val = scaler.transform(X_val)

In [ ]:
batch_size = 2048
train_loader = build_dataloader(X_train, y_train, batch_size, shuffle=True)
valid_loader = build_dataloader(X_val, y_val, batch_size, shuffle=False)

In [ ]:
print(len(train_loader))

In [ ]:
test_df.iloc[:, 1:] = scaler.transform(test_df.iloc[:, 1:])
# output path
output_dir = Path('./', 'output')
output_dir.mkdir(exist_ok=True, parents=True)

In [ ]:
num_epochs = 50
criterion = nn.L1Loss()

In [ ]:
best_epoch_list = []
best_valid_score_list = []

# build model
model = build_model(device, model_name='mlp')
model.to(device)

lr = 0.001
optimizer = AdamW(model.parameters(), lr)

start_time = time()

best_epoch = 0
best_train_loss = 1000
best_valid_score = 1000

for epoch in range(num_epochs):

    model.train()
    optimizer.zero_grad()
    train_loss = 0.0

    for batch_idx, (data, target) in enumerate(train_loader):

        if device:
            data = data.to(device)
            target = target.float().to(device)
        else:
            target = target.float()

        output = model(data)
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()
        train_loss += loss.item() / len(train_loader)

    val_loss, val_score = validation(model, criterion, valid_loader, device)

    elapsed = time() - start_time

    lr = [_['lr'] forz _ in optimizer.param_groups]

    if epoch % 2 == 0:
        print('Epoch {} / {}  train Loss: {:.4f}  val_loss: {:.4f}  val_score: {:.4f}  lr: {:.5f}  elapsed: {:.0f}m {:.0f}s' \
              .format(epoch,  num_epochs - 1, train_loss, val_loss, val_score, lr[0], elapsed // 60, elapsed % 60))
        
    model_path = output_dir / 'best_model.pt'

    if val_score < best_valid_score:
        best_valid_score = val_score
        best_epoch = epoch
        # best model params
        torch.save(model.state_dict(), model_path)


    best_epoch_list.append(best_epoch)
    best_valid_score_list.append(best_valid_score)

In [ ]:
for epoch, score in zip(best_epoch_list, best_valid_score_list):
    print(epoch, score)

In [ ]:
np.mean(best_valid_score_list)

In [ ]:
batch_size = 2048
test_loader = build_dataloader(test_df.iloc[:, 1:].values, Y=None, batch_size=batch_size, shuffle=False)

In [ ]:
model = build_model(device, model_name='mlp')
model.to(device)

model.eval()
# best params setting
model.load_state_dict(torch.load(model_path))

test_preds = np.zeros((len(test_loader.dataset), 4))

with torch.no_grad():
    for batch_idx, data in enumerate(test_loader):
        if device:
            data = data.to(device)
        outputs = model(data)
        test_preds[batch_idx * batch_size:(batch_idx+1) * batch_size] = outputs.detach().cpu().numpy()

In [ ]:
submission = pd.DataFrame({'id': submission_df['id'],
                           'layer_1':test_preds.transpose()[0],
                           'layer_2':test_preds.transpose()[1],
                           'layer_3':test_preds.transpose()[2],
                           'layer_4':test_preds.transpose()[3]})
submission.to_csv('submission.csv', index=False)

submission.head()

In [ ]:
import pandas as pd
import warnings
warnings.filterwarnings('ignore')
train = pd.read_csv('../input/month1/data_mdc01/train.csv')
test = pd.read_csv('../input/month1/data_mdc01/test.csv')

In [ ]:
#독립변수와 종속변수를 분리합니다.
train_X = train.iloc[:,4:]
train_Y = train.iloc[:,:4]
test_X = test.iloc[:,1:]

In [ ]:
import keras
from keras.models import Sequential
from keras.layers import Dense
model = Sequential()
model.add(Dense(units=160, activation='relu', input_dim=226))
model.add(Dense(units=160, activation='relu'))
model.add(Dense(units=160, activation='relu'))
model.add(Dense(units=4, activation='linear'))

In [ ]:
#모델을 컴파일합니다.
model.compile(loss='mae', optimizer='adam', metrics=['mae'])

In [ ]:
#모델을 학습합니다.
model.fit(train_X, train_Y, epochs=20, batch_size=10000, validation_split = 0.05)

In [ ]:
#예측값을 생성합니다.
pred_test = model.predict(test_X)

In [ ]:
#submission 파일을 생성합니다.
sample_sub = pd.read_csv('../input/month1/data_mdc01/sample_submission.csv', index_col=0)
submission = sample_sub+pred_test
submission.to_csv('submission.csv')

In [ ]:
from fastai.tabular.all import *

In [ ]:
path=Path('../input/month1/data_mdc01')

train=pd.read_csv(path/'train.csv')
test=pd.read_csv(path/'test.csv')
submit=pd.read_csv(path/'sample_submission.csv')

In [ ]:
cont_names=[str(i) for i in range(226)]

y_names=['layer_1','layer_2','layer_3','layer_4']
procs=[Normalize]

In [ ]:
data=train.drop(y_names,axis=1)
# prepare
test=test.drop('id',axis=1)

In [ ]:
dls=TabularDataLoaders.from_df(train,path=path,cont_names=cont_names,
                               procs=procs,y_names=y_names)

In [ ]:
dls.show_batch()

In [ ]:
learn=tabular_learner(dls,lr=0.003)

In [ ]:
learn.fit_one_cycle(5)

In [ ]:
learn.validate()

In [ ]:
test_dl=learn.dls.test_dl(test)
pred,_=learn.get_preds(dl=test_dl)

In [ ]:
pred